In [1]:
import gradio as gr

In [2]:
from src.music.mir import MusicInformationRetreiver

mir = MusicInformationRetreiver(weights_path='mir_weights/')

2024-08-20 22:43:48.222354: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.5/lib64
2024-08-20 22:43:48.222384: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[   INFO   ] MusicExtractorSVM: no classifier models were configured by default
2024-08-20 22:43:48.791713: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-20 22:43:48.793864: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dy

In [3]:
from src.video.gen import VideoGenerator

gen = VideoGenerator()

/home/soroush/.venv/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/soroush/.venv/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
/home/soroush/.venv/lib/python3.11/site-packages/diffusers/models/vq_model.py:20: FutureWarning: `VQEncoderOutput` is deprecated and will be removed in version 0.31. Importing `VQEncoderOutput` from `diffusers.models.vq_model` is deprecated and this will be removed in a future version. Please use `from diffusers.models.autoencoder

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.latent_consistency_models.pipeline_latent_consistency_img2img.LatentConsistencyModelImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.latent_consistency_models.pipeline_latent_consistency_text2img.LatentConsistencyModelPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [6]:
import tempfile

def get_segment_durations(music_path):
    segments = mir(
        music_path, 
        segment_threshold=0.1, 
        genre_threshold=0.4, 
        inst_threshold=0.4, 
        add_segment_info=False
    )
    return [seg['duration'] for seg in segments]


def fn(
    prompts, 
    zoom_directions, 
    zoom_factors, 
    rotate_directions,
    rotate_factors,
    move_directions,
    move_factors,
    fps, 
    music_path
):
    prompts = prompts.strip().split('\n')
    zoom_directions = None if zoom_directions == '' else zoom_directions.strip().split(',')
    rotate_directions = None if rotate_directions == '' else rotate_directions.strip().split(',')
    move_directions = None if move_directions == '' else move_directions.strip().split(',')
    zoom_factors = None if zoom_factors == '' else list(map(float, zoom_factors.strip().split(',')))
    rotate_factors = None if rotate_factors == '' else list(map(float, rotate_factors.strip().split(',')))
    move_factors = None if move_factors == '' else list(map(float, move_factors.strip().split(',')))
    
    durations = get_segment_durations(music_path)
    frames = gen.generate(
        durations, 
        style='realistic',
        generation_fps=2,
        final_fps=fps,
        width=512, 
        height=512,
        strength=0.4,
        prompts=prompts, 
        rotate_directions=rotate_directions,
        zoom_directions=zoom_directions,
        move_directions=move_directions,
        zoom_factors=zoom_factors,
        rotate_factors=rotate_factors,
        move_factors=move_factors,
        guidance_scale=8.,
        num_inference_steps=4,
        progress_bar=gr.Progress().tqdm,
        negative_prompt='blurry, fuzzy, low quality, chaotic, poor details, dark, sad, text, letters, alphabet'
    )
    gen.save_video(frames, music_path, 'temp.mp4', fps=fps)
    return 'temp.mp4'
    

In [7]:
demo = gr.Interface(
    fn=fn,
    inputs=[
        gr.Textbox(lines=5, placeholder="Enter prompts separated by enter"),
        gr.Textbox(lines=5, placeholder="Enter zoom directions separated by commas"),
        gr.Textbox(lines=5, placeholder="Enter zoom factors separated by commas"),
        gr.Textbox(lines=5, placeholder="Enter rotation directions separated by commas"),
        gr.Textbox(lines=5, placeholder="Enter rotation factors separated by commas"),
        gr.Textbox(lines=5, placeholder="Enter move directions separated by commas"),
        gr.Textbox(lines=5, placeholder="Enter move factors separated by commas"),
        gr.Slider(minimum=10, step=10, maximum=30, value=20, label="Frames per second"),
        gr.Audio(label="Upload Music File", type='filepath', sources='upload')
    ],
    outputs=[
        gr.Video(label="Generated Video")
    ],
    title="Music Video Generation App",
    description="Generate a music video using Stable Diffusion based on prompts and music input."
)

In [8]:
prompts = [
    "A bustling city street at dawn, with the first rays of sunlight reflecting off tall glass buildings.",
    "The city slowly wakes up as early morning commuters start to fill the streets, and shop signs begin to light up.",
    "Traffic increases, and the sound of honking cars and footsteps fills the air as the city becomes more alive.",
    "As the day progresses, the streets are filled with people, and the city hums with activity under the midday sun.",
    "A bustling marketplace opens, with vendors setting up stalls and people browsing through various goods.",
    "The afternoon brings a slight lull, with some people taking breaks in nearby parks, and the shadows grow longer.",
    "Clouds gather above the city, casting a grayish hue over the buildings, as the wind starts to pick up.",
    "As the evening approaches, the city is bathed in a soft golden light, with lights starting to twinkle in windows and on street signs."
]
print('\n'.join(prompts))

A bustling city street at dawn, with the first rays of sunlight reflecting off tall glass buildings.
The city slowly wakes up as early morning commuters start to fill the streets, and shop signs begin to light up.
Traffic increases, and the sound of honking cars and footsteps fills the air as the city becomes more alive.
As the day progresses, the streets are filled with people, and the city hums with activity under the midday sun.
A bustling marketplace opens, with vendors setting up stalls and people browsing through various goods.
The afternoon brings a slight lull, with some people taking breaks in nearby parks, and the shadows grow longer.
Clouds gather above the city, casting a grayish hue over the buildings, as the wind starts to pick up.
As the evening approaches, the city is bathed in a soft golden light, with lights starting to twinkle in windows and on street signs.


In [9]:
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://4a3f83c7f559adbc59.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
